##Chat Bot

In [3]:
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import trim_messages

### Modellinitialisierung

In [4]:
model = ChatOllama(model="llama3.1")

/var/folders/4h/pxcb698d1zq7fylq212mnvch0000gn/T/ipykernel_10430/1457499772.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  model = ChatOllama(model="llama3.1")


### Stateless Chat Example

In [6]:
stateless_response = model.invoke([HumanMessage(content="Hi! I'm Bob")])
print(stateless_response.content)

followup_response = model.invoke([HumanMessage(content="What's my name?")])
print(followup_response.content)

Hello Bob! What brings you here today? Do you have a question, need some assistance or just want to chat? I'm all ears!
I'm happy to chat with you, but I don't actually know your name. This is our first conversation, and I don't have any information about you. Would you like to introduce yourself?


### Conversation Memory mit Message History

In [7]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

### Prompt Templates

In [8]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your ability in {language}."),
    MessagesPlaceholder(variable_name="messages"),
])
chain = prompt | model

### Kombination: Komplexe Chain + Memory

In [9]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

### Trimmen der History

In [ ]:
trimmer = trim_messages(
    max_tokens=40,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

chain_with_trimmer = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)